In [4]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Metrics
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
# Data generators with rescaling
train_datagen_with_aug = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = train_datagen_with_aug.flow_from_directory(
    r'/Users/hardikchhipa/Desktop/DR-Arvind/Aravind eye dataset/aptos2019-blindness-detection/train_split',
    target_size=(224, 224),  # Resize images to a fixed size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen_with_aug.flow_from_directory(
    r'/Users/hardikchhipa/Desktop/DR-Arvind/Aravind eye dataset/aptos2019-blindness-detection/train_split',
    target_size=(224, 224),  # Resize images to a fixed size
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3298 images belonging to 5 classes.
Found 364 images belonging to 5 classes.


In [7]:
# Functional API for building the model
input_img = Input(shape=(224, 224, 3))
densenet_base = DenseNet201(weights='imagenet', include_top=False, input_tensor=input_img)

# Freeze layers in DenseNet121
for layer in densenet_base.layers:
    layer.trainable = False

# Add custom layers on top of DenseNet121
x = Flatten()(densenet_base.output)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
output = Dense(5, activation='softmax')(x)

# Construct the model
model = Model(inputs=input_img, outputs=output)

# Summary of the model
# model.summary()

# Compile the model
adam = Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', precision_m, recall_m, f1_m])

# Train the model
graph = model.fit(train_generator, epochs=25, validation_data=validation_generator)

# Save the model
model.save("/Users/hardikchhipa/Desktop/DR-Arvind/saved_models/used_models/densenet_121.h5")

# Print the results
print(graph.history)

Epoch 1/25


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-09-05 13:51:33.909278: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 4 of 8
2024-09-05 13:51:44.202110: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


104/104 ━━━━━━━━━━━━━━━━━━━━ 621s 6s/step - accuracy: 0.5644 - f1_m: 0.5664 - loss: 13.9241 - precision_m: 0.5735 - recall_m: 0.5612 - val_accuracy: 0.6813 - val_f1_m: 0.6930 - val_loss: 0.7965 - val_precision_m: 0.7431 - val_recall_m: 0.6510
Epoch 2/25


2024-09-05 14:01:44.794090: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 4 of 8
2024-09-05 14:01:55.729564: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


104/104 ━━━━━━━━━━━━━━━━━━━━ 566s 5s/step - accuracy: 0.7317 - f1_m: 0.7309 - loss: 0.8301 - precision_m: 0.7754 - recall_m: 0.6927 - val_accuracy: 0.7775 - val_f1_m: 0.7600 - val_loss: 0.5902 - val_precision_m: 0.9044 - val_recall_m: 0.6589
Epoch 3/25


...

104/104 ━━━━━━━━━━━━━━━━━━━━ 568s 5s/step - accuracy: 0.8681 - f1_m: 0.8678 - loss: 0.3326 - precision_m: 0.9253 - recall_m: 0.8190 - val_accuracy: 0.7857 - val_f1_m: 0.7885 - val_loss: 0.6851 - val_precision_m: 0.8253 - val_recall_m: 0.7569
Epoch 19/25


2024-09-05 16:42:32.759490: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 4 of 8
2024-09-05 16:42:43.047261: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


104/104 ━━━━━━━━━━━━━━━━━━━━ 1783s 17s/step - accuracy: 0.8471 - f1_m: 0.8586 - loss: 0.3742 - precision_m: 0.9172 - recall_m: 0.8092 - val_accuracy: 0.7995 - val_f1_m: 0.7815 - val_loss: 0.6993 - val_precision_m: 0.8210 - val_recall_m: 0.7474
Epoch 20/25


2024-09-05 17:12:15.795853: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 5 of 8
2024-09-05 17:12:23.819869: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


 18/104 ━━━━━━━━━━━━━━━━━━━━ 27:05 19s/step - accuracy: 0.8511 - f1_m: 0.8820 - loss: 0.3726 - precision_m: 0.9308 - recall_m: 0.8402